In [ ]:
%load_ext autoreload
%autoreload 2

# Segmentation

## Règles

_Avant de s’interroger sur les responsabilités d’un algorithme, il faut être en mesure d’identifier quel algorithme est la source de telle décision. Ainsi, en parallèle des conférences, la société civile est invitée dans le cadre d’un Hackathon, à contribuer à la résolution de ce problème ouvert via des techniques d’intelligence artificielle._
_Le Hackathon propose un cas simplifié, mais réaliste, de plateforme de livraison de plats cuisinés. La plateforme affiche à l’utilisateur un tarif unique pour chaque livraison, depuis un restaurant jusqu’au lieu de collecte. Plusieurs options sont possibles pour l'établissement de ce tarif : le restaurant décide quelle option de livraison et/ou tarification de livraison s’applique à lui parmi différents algorithmes proposés par la plateforme ou la tarification est complètement à la main du restaurant._

_Pour comprendre les responsabilités à l’œuvre, les participants inscrits auront pour mission de détecter à partir du jeu de données fourni quel algorithme fixe quelle tarification. Afin d’aider les participants, le jeu de données contient des informations récupérées sur la plateforme fictive (e.g. le lieu de livraison) comme des données contextuelles (e.g. des informations sur la météo locale). Toute donnée supplémentaire apportée par les candidats est bienvenue._

## Méthodologie

## Objectif visé
Déterminer, pour chaque commande, le mode de tarification utilisé (variable caché) étant donné le tarif affiché à l'utilisateur (qui est notre observable).

### Données utilisées (et sources)
Météo (jdd fourni)
Géolocalisation des points de livraison (jdd fourni)
Géolocalisation des commerces (jdd fourni)
Jour et heure (jdd fourni)

### Spécifications techniques du code
python3.10
sklearn==1.1.0

# Feature engineering

In [ ]:
import pandas as pd
tarifs = pd.read_csv("tarifs.csv")

In [ ]:
tarifs.head()

#### Adding restaurant id.
#### Also addd addresses (?) (seem to be useless. Maybe the addressed are anonymized ?)

In [ ]:
tarifs["id"] = tarifs.apply(lambda row: str(row["start_lat"]) + str(row["start_lon"]), axis=1)

In [ ]:
"""
tarifs["lat"] = tarifs["start_lat"]
tarifs["lon"] = tarifs["start_lon"]
tarifs.to_csv("tarifs_geo.csv")
!curl -X POST -F data=@tarifs_geo.csv https://api-adresse.data.gouv.fr/reverse/csv/ > tarifs_adresses.csv
"""


### Distance

In [ ]:
import numpy as np

In [ ]:
# Création de la colonne "distance au point de livraison"
tarifs["d"] = np.sqrt((tarifs["end_lon"]-tarifs["start_lon"])**2 + (tarifs["end_lat"]-tarifs["start_lat"])**2)

## Arrondissement

In [ ]:
# it works because the last 2 digits of code INSEE equals the arrondissement number for Paris
tarifs["arrondissement"] = tarifs.apply(lambda row: str(row["end_code_postal"])[-2:], axis=1)

## Jour

In [ ]:
day_dict = {"Monday": 1, "Tuesday": 2, "Wednesday": 3, "Thursday": 4, "Friday": 5, "Saturday": 6, "Sunday": 7}
tarifs["day_code"] = tarifs.apply(lambda row: day_dict[row["day_of_week"]], axis=1)
df = tarifs[["id", "rain", "heat", "day_code", "hour", "fee", "d", "arrondissement", "observation_uuid", "minute"]]
df.head(10)

In [ ]:
from sklearn.model_selection import train_test_split
## à faire plus tard avant de soumettre

In [ ]:
# Nos idées, à mettre plus tard dans le README.md (à la fin du hackathon)

### Modèle constant (mod 1) et modèle rain - deux valeurs selon qu'on est  rain>=6 ou non  (outdated - mod 2)

In [ ]:
df["rain_thresh"] = 0
df["heat_thresh"] = 0

df.loc[df["rain"]>=6,"rain_thresh"]=1 # values 6, 7, 8, 9
df.loc[df["heat"]<=3,"heat_thresh"]=1 # values 0, 1,2, 3
#df.loc[df["heat"]>=8,"heat_thresh"]=1 # values 8,9


In [ ]:
def get_ratios(distances, fees, rains, heats):
    d_calm, d_notcalm = 0, 0
    f_calm, f_notcalm = 0, 0
    for k, _ in enumerate(distances):
        if rains[k] == 1 or heats[k] == 1: # not calm
            d_notcalm += distances[k]
            f_notcalm += fees[k]
        else:
            d_calm += distances[k]
            f_calm += fees[k]
    if not d_calm == 0 and not d_notcalm == 0:
        ratio_calm = f_calm / d_calm
        ratio_notcalm = f_notcalm / d_notcalm
        error = np.abs(ratio_calm - ratio_notcalm)/ratio_calm
        return error
    else:
        
        return 0
    

In [ ]:
grouped = df[["id", "d", "fee", "rain_thresh", "heat_thresh"]].groupby("id").agg(list)

grouped["weather_sensible"] = grouped.apply(
    lambda row: get_ratios(row["d"], row["fee"],row["rain_thresh"], row["heat_thresh"]),
    axis=1
)
df = pd.merge(df, grouped[["weather_sensible"]].reset_index(), left_on="id", right_on="id")

In [ ]:
df["weather_thresh"] = 0
df.loc[df["weather_sensible"] > 0.22, "weather_thresh"] = 1


## Week day

In [ ]:
def get_ratios_weekends(distances, fees, days): # Friday and weekends
    d, d_not = 0, 0
    f, f_not = 0, 0
    for k, day in enumerate(days):
        if day>=5: # friday, sunday, saturday
            d += distances[k]
            f += fees[k]
        else:
            d_not += distances[k]
            f_not += fees[k]
    if not d == 0 and not d_not == 0:
        ratio_calm = f / d
        ratio_notcalm = f_not / d_not
        error = np.abs(ratio_calm - ratio_notcalm)/ratio_calm
        return error
    else:
        
        return 0



In [ ]:
grouped = df[["id", "d", "fee", "day_code"]].groupby("id").agg(list)

grouped["day_sensible"] = grouped.apply(
    lambda row: get_ratios_weekends(row["d"], row["fee"],row["day_code"]),
    axis=1
)
df = pd.merge(df, grouped[["day_sensible"]].reset_index(), left_on="id", right_on="id")

In [ ]:

restaurant_model = df[["fee", "id"]].groupby("id").nunique().reset_index()
restaurant_model["model"] = 0
restaurant_model.rename(columns={"fee":"nb_fee"}, inplace=True)
restaurant_model

In [ ]:
df = pd.merge(df, restaurant_model[["id", "model", "nb_fee"]], left_on="id", right_on="id")

In [ ]:
import seaborn as sns
sns.pairplot(df[["fee", "d", "weather_thresh"]][df.nb_fee!=1], hue="weather_thresh", kind="hist", diag_kind="kde")#, palette="id") #, hue="col1", palette="col2")

# TODO: regarder la série temporelle de la tarification moyenne ?

### Pearson correlation (fee, distance)

In [ ]:
from scipy.stats import pearsonr

grouped = df[["id", "d", "fee"]].groupby("id").agg(list)
grouped["corr_d"] = grouped.apply(
    lambda row: pearsonr(x=row["fee"], y=row["d"])[0],axis=1
)

df = pd.merge(df, grouped[["corr_d"]].reset_index(), left_on="id", right_on="id")


In [ ]:
CORR_D_THRESH = 0.9
df["corr_d_thresh"] = 0
df.loc[df["corr_d"] > CORR_D_THRESH, "corr_d_thresh"] = 1

In [ ]:
df.corr_d_thresh.value_counts()

### Pearson correlation (fee, hour)

In [ ]:
from scipy.stats import pearsonr
df["t"] = df["hour"]*60+df["minute"]
grouped = df[["id", "t", "fee"]].groupby("id").agg(list)
grouped["corr_t"] = grouped.apply(
    lambda row: pearsonr(x=row["fee"], y=row["t"])[0],axis=1
)

df = pd.merge(df, grouped[["corr_t"]].reset_index(), left_on="id", right_on="id")


In [ ]:
import seaborn as sns

sns.histplot(df["weather_sensible"],bins=101)

In [ ]:
CORR_T_UPPER_THRESH = 0.3
CORR_T_LOWER_THRESH = -0.3
df["corr_t_thresh"] = 0
df.loc[(df["corr_t"] > CORR_T_UPPER_THRESH) | (df["corr_t"] < CORR_T_LOWER_THRESH), "corr_t_thresh"] = 1


In [ ]:
df[["nb_fee","weather_thresh", "weather_sensible", "day_sensible","observation_uuid", "corr_d", "corr_t", "t"]].to_csv("results.csv")

## Soumission generation

In [ ]:
import pandas as pd
#import numpy as np

df = pd.read_csv("results.csv")

def f(row):
    if row["nb_fee"] == 1:
        return 1
    if row["corr_d"] >= 0.88:
        return 2
    if row["weather_sensible"] <= 0.22:
        return 3
    return 4

df["algorithm"] = df.apply(f, axis=1)

soumission = df[["observation_uuid", "algorithm"]]
n = 1
soumission.to_csv(f"soumission_{n}.csv", index=False)

# What would a linear estimate mixing all commands from all restaurants ?

In [ ]:
from sklearn.cluster import KMeans
from sklearn.linear_model import Lasso, Ridge, RidgeCV, LassoCV
from sklearn.kernel_ridge import KernelRidge
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

model = RidgeCV(cv=20)

std = StandardScaler()
pip = make_pipeline(
    std, 
    lasso_
)

X, Y=df[["d"]][df.nb_fee!=1], df["fee"][df.nb_fee!=1]
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.5)
pip.fit(X_train, Y_train)

ABS_DIST_LINEAR = 0.4
rather_linear = (np.abs(pip.predict(X)-Y)<ABS_DIST_LINEAR)
df["LINEAR"] = rather_linear
df

## Visualisations gridplot

In [ ]:
"""
import plotly.express as px
counts, bins = np.histogram(df_annotated.fee, bins=301)
bins = 0.5 * (bins[:-1] + bins[1:])

fig = px.bar(x=bins, y=counts, labels={'x':'fee', 'y':'count'})
fig.show()
"""

In [ ]:
import seaborn as sns
import plotly # https://plotly.com/python/

sns.pairplot(df[["rain", "nb_fee", "fee", "d", "heat", "weather_thresh"]], hue="weather_thresh", kind="hist", diag_kind="kde")#, palette="id") #, hue="col1", palette="col2")
# visualisation 2D à creuser : (distance OU temps OU nb course) x commmission 
# notion éventuelle de bonus pour certaines courses à certains moments ou certains endroits 
# bonus : (regarder la série temporelle de la tarification moyenne), ou la distribution géographique

In [ ]:
import seaborn as sns
import plotly # https://plotly.com/python/

sns.pairplot(df[["rain", "model", "fee", "t", "d"]], hue="rain", kind="hist", diag_kind="kde", palette=sns.color_palette("Set2", 10))

### Modélisation avec des ellipses pour gérer le bruit sur la distance (n'a rien donné...)

In [ ]:
from sklearn.mixture import GaussianMixture
from sklearn.mixture import BayesianGaussianMixture
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
"""
model = GaussianMixture(n_components=3, covariance_type="full")
model = BayesianGaussianMixture(n_components=3)

pip = make_pipeline(StandardScaler(), model)

X_train, X_test = train_test_split(X, test_size=0.25)
pip.fit(X_test)
"""
# https://scikit-learn.org/dev/modules/mixture.html#mixture
# make_pipeline(StandardScaler(), model)